# Install conda on your Colab environment

Ignore this first cell if you are running the notebook in a local environment.

One can still run it locally but it will have no effect.

In [1]:
# Run this cell first - it will install a conda distribution (mamba)
# on your Drive then restart the kernel automatically 
# (don't worry about the crashing/restarting kernel messages)
# It HAS to be runned FIRST everytime you use the notebook in colab

import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    !pip install -q condacolab
    import condacolab
    condacolab.install()

# Set up your Colab or local environment
# Then import libraries

Run this cell in both cases of use (local or Colab)

In [2]:
import os
import sys
RunningInCOLAB  = 'google.colab' in str(get_ipython())

if RunningInCOLAB:
    
    # Check everything is fine with conda in Colab
    import condacolab
    condacolab.check()
    
    # Mount your drive environment in the colab runtime
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    
    # Change this variable to your path on Google Drive to which the repo has been cloned
    # If you followed the colab notebook 'repo_cloning.ipynb', nothing to change here
    repo_path_in_drive = '/content/drive/My Drive/Github/amn_release/'
    # Change directory to your repo cloned in your drive
    DIRECTORY = repo_path_in_drive
    os.chdir(repo_path_in_drive)
    # Copy the environment given in the environment_amn_light.yml
    !mamba env update -n base -f environment_amn_light.yml
    
    # This is one of the few Colab-compatible font
    font = 'Liberation Sans'
    
else:
    
    # In this case the local root of the repo is our working directory
    DIRECTORY = './'
    font = 'arial'

# printing the working directory files. One can check you see the same folders and files as in the git webpage.
print(os.listdir(DIRECTORY))

from Library.Build_Model import *

# We declare this function here and not in the
# function-storing python file to modify it easily
# as it can change the printouts of the methods
def printout(filename, Stats, model, time): 
    # printing Stats
    if Stats == None:
        print('Stats for %s failed CPU-time %.4f' % (filename, time))
        return
    print('Stats for %s CPU-time %.4f' % (filename, time))
    print('R2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.train_objective[0], Stats.train_objective[1],
           Stats.train_loss[0], Stats.train_loss[1]))
    print('Q2 = %.4f (+/- %.4f) Constraint = %.4f (+/- %.4f)' % \
          (Stats.test_objective[0], Stats.test_objective[1],
           Stats.test_loss[0], Stats.test_loss[1]))
    
# Get R2/Q2 and constraints
def collate_stats(model, parameter, measurement, Y, verbose=False):  
    if verbose: print(Y.shape, parameter.Y.shape)
    Y_true = parameter.Y[:, measurement]
    Y_pred = Y[:, measurement]
    RQ2 = r2_score(Y_true, Y_pred, multioutput='variance_weighted')
    if verbose: print('RQ2 =', RQ2)
    X = tf.convert_to_tensor(np.float32(model.X)) # Loss computed of tf tensors
    Y = tf.convert_to_tensor(np.float32(Y))
    L1 = (np.square(Y_true - Y_pred)).mean(axis=0)
    if verbose: print('Loss_Vout =', L1)
    L2, _ = Loss_SV(Y, model.S)
    L2 = np.mean(L2.numpy())
    if verbose: print('Loss_SV =', L2)
    L3, _ = Loss_Vin(Y, model.Pin, X, model.mediumbound)
    L3 = np.mean(L3.numpy())
    if verbose: print('Loss_Vin =', L3)
    L = (L1+L2+L3)/3
    if verbose: print('Constraints =', L)
    return RQ2, L

def collate_stats_RF(model, Y_ref, measurement, Y_preds, verbose=False):  
    if verbose: print(Y_preds.shape, Y_ref.shape)
    Y_true = Y_ref[:, measurement]
    Y_pred = Y_preds[:, measurement]
    Q2 = r2_score(Y_true, Y_pred)
    if verbose: print('RQ2 =', Q2)
    X = tf.convert_to_tensor(np.float32(model.X)) # Loss computed of tf tensors
    Y = tf.convert_to_tensor(np.float32(Y_preds))
    L1 = (np.square(Y_true - Y_pred)).mean(axis=0)
    if verbose: print('Loss_Vout =', L1)
    L2, _ = Loss_SV(Y, model.S)
    L2 = np.mean(L2.numpy())
    if verbose: print('Loss_SV =', L2)
    L3, _ = Loss_Vin(Y, model.Pin, X, model.mediumbound)
    L3 = np.mean(L3.numpy())
    if verbose: print('Loss_Vin =', L3)
    L = (L1+L2+L3)/3
    if verbose: print('Constraints =', L)
    return Q2, L

['README.md', 'Duplicate_Model.ipynb', 'Dataset_experimental', 'Tutorial.ipynb', '.ipynb_checkpoints', '.git', 'Build_Model_RC.ipynb', 'biolog_simulations.npy', 'environment_amn_light.yml', 'Build_Experimental.ipynb', 'Reservoir', 'Build_Model_MM.ipynb', 'Dataset_model', 'Figures.ipynb', 'Result', 'Figures', '.gitignore', 'LICENSE', 'Build_Model_ANN_Dense.ipynb', 'Build_Dataset.ipynb', 'Build_Model_RF.ipynb', 'Library', 'Dataset_input', 'Functions', 'environment_amn.yml', 'Build_Model_AMN.ipynb', '.DS_Store']


# Create and Train Random Forest

In [3]:
# What you can change
seed = 20
np.random.seed(seed=seed)

# trainname = 'e_coli_core_UB_10'
# trainname = 'e_coli_core_EB_10'
# trainname = 'e_coli_core_UB_50'
# trainname = 'e_coli_core_UB_100'
# trainname = 'e_coli_core_UB_500'
trainname = 'e_coli_core_UB_1000'
# End of what you can change

# Load training set
trainingfile = DIRECTORY+'Dataset_model/' + trainname
cobramodel = cobra.io.read_sbml_model(trainingfile+'.xml')
parameter = TrainingSet()
parameter.load(trainingfile)
RQ2, Loss = [], []

model = Neural_Model(trainingfile = trainingfile,
                        model_type = 'ANN_Dense',
                        scaler=True,
                        n_hidden = 1, hidden_dim = 50,
                        epochs = 500, xfold = 5)

model.X.shape, model.Y.shape

((1000, 20), (1000, 154))

In [4]:
# train a Random Forest model

from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict as cvp
from sklearn.metrics import r2_score

cobramodel = cobra.io.read_sbml_model(trainingfile+'.xml')
biomass_index = get_index_from_id('BIOMASS_Ecoli_core_w_GAM',cobramodel.reactions)

xfold = 5
Maxloop = 3

RQ2, Loss = [], []

for Nloop in range(Maxloop):

    RF = MultiOutputRegressor(rfr(n_estimators=1000, max_depth=None, random_state=Nloop))
        
    rf_pred = cvp(RF, model.X, model.Y, cv=KFold(n_splits=xfold, shuffle=True, random_state=Nloop), n_jobs = xfold)

    rq2, l = collate_stats_RF(model, rf_pred, biomass_index, model.Y, verbose=True)
    RQ2.append(rq2)
    Loss.append(l)

# Print stats averaged over all iterations
rqt = 'R2 (biomass)' if xfold < 2 else 'Q2 (biomass)'
print(rqt, '= %.4f (+/- %.4f) Loss = %.4f (+/- %.4f)' \
      % (np.mean(RQ2), np.std(RQ2), np.mean(Loss), np.std(Loss)))

(1000, 154) (1000, 154)
RQ2 = 0.9559045499768442
Loss_Vout = 0.000678624571590675
Loss_SV = 1.5949273e-07
Loss_Vin = 0.0
Constraints = 0.00022626135477301092
(1000, 154) (1000, 154)
RQ2 = 0.9564431916152767
Loss_Vout = 0.0006736933675567493
Loss_SV = 1.5949273e-07
Loss_Vin = 0.0
Constraints = 0.00022461762009503566
(1000, 154) (1000, 154)
RQ2 = 0.9561115766646224
Loss_Vout = 0.0006665711439733366
Loss_SV = 1.5949273e-07
Loss_Vin = 0.0
Constraints = 0.00022224354556723145
Q2 (biomass) = 0.9562 (+/- 0.0002) Loss = 0.0002 (+/- 0.0000)


In [4]:
# Some exploration of the results

# R2 on all fluxes, variance weighted or not
print(r2_score(model.Y, rf_pred, multioutput='variance_weighted'), r2_score(model.Y, rf_pred, multioutput='uniform_average'))
# print(r2_score(model.Y, rf_pred, multioutput='raw_values'))

# Some fluxes make the R2 scoring bad because they have always the same value in training set
# Examples: 15 is for ATPM, always at 8.39 (lower bound value); 72 is an uptake which is always at 10
print(r2_score(model.Y[:,15], rf_pred[:,15]))

# Random Forests can predict a constant value, ANN has troubles for that
print(model.Y[:,15], rf_pred[:,15])

# If looking at the MSE, it makes more sense (R2 is probably not the best metric when having constant fluxes in true)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(model.Y[:,15], rf_pred[:,15]))

0.5587182693370569 -0.4053601982471869
0.0
[8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39
 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39
 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39
 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39] [8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39
 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39
 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39
 8.39 8.39 8.39 8.39 8.39 8.39 8.39 8.39]
1.2129525278678278e-26


In [ ]:
"""
e_coli_core_UB_10
RQ2 = -0.9614889870359049
Loss_Vout = 0.007205661434244432
Loss_SV = 1.341026e-07
Loss_Vin = 0.0
Constraints = 0.002401931845616928
Q2 (biomass) = -0.9294 (+/- 0.0405) Loss = 0.0026 (+/- 0.0002)

e_coli_core_EB_10
RQ2 = 0.46710287005289863
Loss_Vout = 0.003945400312211618
Loss_SV = 1.341026e-07
Loss_Vin = 0.0
Constraints = 0.0013151781382726567
Q2 (biomass) = 0.4360 (+/- 0.0616) Loss = 0.0015 (+/- 0.0002)

e_coli_core_UB_50
RQ2 = 0.5042871268837632
Loss_Vout = 0.0044485842385107366
Loss_SV = 1.4163145e-07
Loss_Vin = 0.0
Constraints = 0.0014829086233189544
Q2 (biomass) = 0.5402 (+/- 0.0274) Loss = 0.0014 (+/- 0.0001)

e_coli_core_UB_100
RQ2 = 0.7412160739010045
Loss_Vout = 0.002787731042068949
Loss_SV = 1.4716305e-07
Loss_Vin = 0.0
Constraints = 0.0009292927350395651
Q2 (biomass) = 0.7776 (+/- 0.0259) Loss = 0.0008 (+/- 0.0001)

e_coli_core_UB_500
RQ2 = 0.9295530992884391
Loss_Vout = 0.0010030032828992294
Loss_SV = 1.58766e-07
Loss_Vin = 0.0
Constraints = 0.00033438734963289594
Q2 (biomass) = 0.9314 (+/- 0.0025) Loss = 0.0003 (+/- 0.0000)
"""